<a href="https://colab.research.google.com/github/Sruthi-sk/MultiDeepRLNeuroGambling/blob/main/1_get_fmri_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load HCP parcellated task data 
## (version with Behavioural Data)

The HCP dataset comprises task-based fMRI from a large sample of human subjects. The NMA-curated dataset includes time series data that has been preprocessed and spatially-downsampled by aggregating within 360 regions of interest.

In order to use this dataset, please electronically sign the HCP data use terms at [ConnectomeDB](https://db.humanconnectome.org). Instructions for this are on pp. 24-25 of the [HCP Reference Manual](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf).

In this notebook, NMA provides code for downloading the data and doing some basic visualisation and processing.

In [ ]:
# @title Install dependencies
!pip install nilearn --quiet

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [ ]:
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'
# TIP: look inside the data folders for more fine-graned conditions
EXPERIMENTS = {
    'GAMBLING'   : {'cond':['loss','win','neut']}
}

> For a detailed description of the tasks have a look pages 45-54 of the [HCP reference manual](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf).

# Downloading data

The task data are shared in different files, but they will unpack into the same directory structure.


In [ ]:
# @title Download data file
import os, requests

fname = "hcp_task.tgz"
url = "https://osf.io/2y3fw/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp_task"

# importing the "tarfile" module
import tarfile

# open file
with tarfile.open(fname) as tfile:
  # extracting file
  tfile.extractall('.')

subjects = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')

## Understanding the folder organisation

The data folder has the following organisation:

- hcp
  - regions.npy (information on the brain parcellation)
  - subjects_list.txt (list of subject IDs)
  - subjects (main data folder)
    - [subjectID] (subject-specific subfolder)
      - EXPERIMENT (one folder per experiment)
        - RUN (one folder per run)
          - data.npy (the parcellated time series data)
          - EVs (EVs folder)
            - [ev1.txt] (one file per condition)
            - [ev2.txt]
            - Stats.txt (behavioural data [where available] - averaged per run)
            - Sync.txt (ignore this file)



## Loading region information

Downloading this dataset will create the `regions.npy` file, which contains the region name and network assignment for each parcel.

Detailed information about the name used for each region is provided [in the Supplement](https://static-content.springer.com/esm/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_BFnature18933_MOESM330_ESM.pdf) to [Glasser et al. 2016](https://www.nature.com/articles/nature18933).

Information about the network parcellation is provided in [Ji et al, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6289683/).



In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

In [ ]:
regions[:,20:30]

array([['R_LO2', 'R_PIT', 'R_MT', 'R_A1', 'R_PSL', 'R_SFL', 'R_PCV',
        'R_STV', 'R_7Pm', 'R_7m'],
       ['Visual2', 'Visual2', 'Visual2', 'Auditory', 'Default',
        'Default', 'Dorsal-atten', 'Default', 'Frontopariet',
        'Posterior-Mu'],
       ['1.93288', '1.9352', '2.03289', '2.29365', '1.77188', '1.68062',
        '1.85286', '1.83748', '1.8366', '1.91503']], dtype='<U12')

In [ ]:
first_key = list(region_info)[1]
first_val = list(region_info.values())[1]
print(first_key,":",first_val[-5:])

network : ['Posterior-Mu' 'Frontopariet' 'Cingulo-Oper' 'Cingulo-Oper'
 'Cingulo-Oper']


# Help functions

We provide two helper functions: one for loading the time series from a single suject and a single run, and one for loading an EV file for each task. 

An EV file (EV:Explanatory Variable) describes the task experiment in terms of stimulus onset, duration, and amplitude. These can be used to model the task time series data.

In [ ]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


In [ ]:
def load_df_evs_frames(subject, experiment, run):
  """Load EVs (explanatory variables) data for one task experiment.
  takes from reward, loss and neutral trials

  Args:
    subject (str): subject ID to load
    experiment (str) : Name of experiment
    run (int): 0 or 1

  Returns
    evs (list of lists): A list of frames associated with each condition

  """
  events_df = pd.DataFrame()

  task_key = f'tfMRI_{experiment}_{RUNS[run]}'
  for cond in EXPERIMENTS[experiment]['cond']:
    #print(cond)
    frames_list = []
    ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}_event.txt"
    #print(ev_file)
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    #print(ev)
    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)
    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)
    #print("Sample start,duration : ",start,duration)
    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
    #frames_list.append(frames)
    for fr in frames:
      st = fr[0]
      events_df=events_df.append({'condition':cond,'Timepoints':fr,'start':st}, ignore_index=True)

  return events_df

# Example run

Let's load the timeseries data for the GAMBLING experiment from a single subject and a single run

In [ ]:
my_exp = 'GAMBLING'
my_subj = subjects[1]
my_run = 1

data = load_single_timeseries(subject=my_subj,
                              experiment=my_exp,
                              run=my_run,
                              remove_mean=True)
print(data.shape)

(360, 253)


In [ ]:
print(data.shape[1]*TR, " seconds")

182.16  seconds


As you can see the time series data contains 253 time points in 360 regions of interest (ROIs).

np.ceil(28 / TR).astype(int) # 28 second took 39 samples. #3.5 secs took 5 samples


In [ ]:
evs = load_df_evs_frames(subject=my_subj, experiment=my_exp, run=my_run)
evs_sorted = evs.sort_values('start')
evs_sorted.drop(columns=['start'],inplace=True)
evs_sorted.reset_index(inplace=True,drop=True)

loss
Sample start,duration :  [ 21  77  82  87  97 102 107 133 138 143 153 163 168 203 213] [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
win
Sample start,duration :  [ 11  16  26  31  36  46  72 193 198 208 219 224 229] [5 5 5 5 5 5 5 5 5 5 5 5 5]
neut
Sample start,duration :  [ 41  92 148 158] [5 5 5 5]


In [ ]:
time_fmri_data = data.transpose()
print(time_fmri_data.shape)
evs_sorted.iloc[5]['Timepoints']

(253, 360)


array([36, 37, 38, 39, 40])

In [ ]:
evs_with_fmri = evs_sorted.copy()
empty_fmri = [np.zeros(360*5)]*evs_sorted.shape[0]
empty_fmri = [np.zeros(360*5)]*evs_sorted.shape[0]
evs_with_fmri['fmri']=empty_fmri

In [ ]:
# Create mega array of (No of subjects=1 , Condition , Features (5*360) )
no_of_events = evs_sorted.shape[0]
for i in range(no_of_events): #no_of_events
  frame= evs_sorted.iloc[i]['Timepoints']
  fmri_frame = time_fmri_data[frame,:]
  fmri_frame_flat=np.ravel(fmri_frame)
  evs_with_fmri.loc[i]['fmri']=fmri_frame_flat

In [ ]:
evs_with_fmri.drop(columns='Timepoints',inplace=True)

In [ ]:
evs_with_fmri

,condition,fmri
0,win,"[-70.0626028521965, 17.179175551278604, 92.596..."
1,win,"[-202.47391434957535, -14.862366903489601, -15..."
2,loss,"[-68.27319415024249, -41.45556437470805, -18.9..."
3,win,"[28.0382771708355, 29.212108836640255, 79.1204..."
4,win,"[-39.5773015448176, -8.759275312208047, -25.84..."
5,win,"[-79.7329646146245, -54.77425783687795, -36.78..."
6,neut,"[-28.07082483837803, -84.38374848243802, -23.0..."
7,win,"[-27.201017867681003, -42.815723194609745, 13...."
8,win,"[-21.730672112440516, -65.746014621418, 29.223..."
9,loss,"[48.0615998906178, -12.149052244772975, 102.37..."


# FMRI for all subjects 
in shape (N_subjects,conditions, n_features*timepoints)

In [ ]:
r=0
my_exp = 'GAMBLING'
df_events = pd.DataFrame()
df_fmri = pd.DataFrame()

for s in subjects:
  data = load_single_timeseries(subject=s, experiment=my_exp, run=r, remove_mean=True)
  data=data.transpose()
  evs = load_df_evs_frames(subject=s, experiment=my_exp, run=r)
  evs_sorted = evs.sort_values('start')
  evs_sorted.drop(columns=['start'],inplace=True)
  evs_sorted.reset_index(inplace=True,drop=True)

  evs_fmri = pd.DataFrame()
  # empty_fmri = [np.zeros(360*5)]*evs_sorted.shape[0]
  # evs_fmri['fmri']=empty_fmri
  no_of_events = evs_sorted.shape[0]
  # load fmri data to df
  for i in range(no_of_events):  
    frame= evs_sorted.iloc[i]['Timepoints']
    fmri_frame = data[frame,:]
    fmri_frame_flat=pd.DataFrame(np.ravel(fmri_frame)).transpose()
    evs_fmri=pd.concat([evs_fmri,fmri_frame_flat])
    evs_fmri.reset_index(inplace=True,drop=True)
  evs_sorted.drop(columns='Timepoints',inplace=True)

  #print(evs_sorted.shape,evs_fmri.shape)
  df_events=df_events.append(evs_sorted)
  df_fmri=df_fmri.append(evs_fmri)

In [ ]:
df_events.shape

(3200, 1)

In [ ]:
print(df_fmri.shape)

(3200, 1800)


In [ ]:
df_fmri.iloc[33]

0        20.526074
1       -39.517628
2         1.415378
3       -44.652678
4         4.379296
           ...    
1795     84.472765
1796    -78.427786
1797    -62.015464
1798    176.681714
1799     50.832478
Name: 1, Length: 1800, dtype: float64

In [ ]:
# df_fmri.to_csv('file1.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  file1.csv  hcp_task/  hcp_task.tgz  sample_data/


In [ ]:
cd '/content/drive/My Drive/NMA-DL 2022/nmaproject'

/content/drive/.shortcut-targets-by-id/1lr36-u30LBtfXLdR-6EEp0ha2-OTHhlG/nmaproject


In [ ]:
ls

 1.get_fmri_time_series.ipynb   fmri_transformer.ipynb
 BolT/                         'get_win_loss_reward_trials '
 connectivity_hcp               Input-Cell-Attention/
 fmri_baseline.ipynb           'IO of transformers.ipynb'
 fmri_rnn.ipynb                 original_data


In [ ]:
df_fmri.to_csv('data_fmri_all.csv')

In [ ]:
df_events.to_csv('data_events_all.csv')

In [ ]:
ls

 1.get_fmri_time_series.ipynb   fmri_rnn.ipynb
 BolT/                          fmri_transformer.ipynb
 connectivity_hcp              'get_win_loss_reward_trials '
 events_all.csv                 Input-Cell-Attention/
 fmri_all.csv                  'IO of transformers.ipynb'
 fmri_baseline.ipynb            original_data


In [ ]:
df_fmri

,0,1,2,3,4,5,6,7,8,9,...,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799
0,-38.199091,42.972339,23.782777,-21.746358,-25.364060,-56.569161,-88.103271,24.002839,23.678154,-21.755606,...,-184.555471,-46.214629,21.017382,-113.128886,-28.258001,-5.063690,-65.811263,-24.764323,24.887710,70.467867
1,-19.408738,40.798010,-43.587189,-36.258554,28.598514,4.321796,-36.709137,20.255197,22.256466,11.948283,...,-140.382551,-51.175064,-77.264748,-140.341340,-33.973469,-40.327892,-56.276677,-106.763887,0.516792,-2.871168
2,34.790439,68.129013,-46.958291,17.017267,22.692233,-7.855621,-100.486892,-9.186491,3.720721,-19.217364,...,-106.758232,-126.441789,-23.498876,4.934218,-39.752704,-82.006179,11.808902,-77.285650,-7.289299,21.853963
3,-9.253665,-0.636214,3.102502,11.351100,18.267102,12.589415,31.406105,-22.193624,3.892748,20.695907,...,-107.788097,-108.517859,-47.480590,-20.730076,-62.235035,-67.055212,0.544794,-122.031503,-21.802361,136.615648
4,16.328306,-15.505585,-48.035146,15.002931,-16.396359,-4.455515,-12.811370,-27.342111,-3.645424,-12.477021,...,-75.575259,-109.400285,-75.219268,-90.182379,-51.310730,-53.819095,-36.769716,-80.992343,-25.060227,136.713523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,-34.804387,-19.555912,-52.463524,-66.736259,-73.475935,-80.187007,-7.052904,-10.849052,-31.877924,-3.864255,...,10.557261,-52.891488,23.063001,49.519895,67.345272,7.431189,34.562969,86.804617,-84.416144,16.810562
28,-31.696290,13.541269,-27.105808,-79.862206,-18.085045,-39.646956,-14.865846,10.248980,-4.187231,-9.325255,...,28.484443,-76.117682,-5.670301,67.982979,71.369368,-41.736855,-29.853186,19.502228,74.094048,-24.010693
29,45.524560,24.105273,77.308532,11.921680,32.595728,27.269427,29.772597,-6.538728,-22.307073,28.916619,...,50.995670,-50.873101,-15.176246,3.040005,67.015876,111.744689,4.299122,-62.687225,-80.867843,22.820749
30,32.379549,-5.364415,-19.693826,-13.014656,-35.750181,-44.808175,-100.937975,8.108065,-16.028607,-10.227932,...,2.567769,-21.573245,81.446329,13.286214,36.108484,-31.531012,10.083105,146.221857,-46.056464,41.314132


In [ ]:
df_events

,condition
0,loss
1,loss
2,win
3,loss
4,win
...,...
27,win
28,loss
29,win
30,win


In [ ]:
df_events.iloc[60]

condition    loss
Name: 28, dtype: object